In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('summary_2022.csv', encoding='utf-8-sig')

def read_file(year):
    PATH = f'imbank_customer_data/iMBANK_CUSTOMER_DATA_{year}(K-DigitalTraining).csv'
    df = pd.read_csv(PATH, encoding='euc-kr', usecols=['기준년월', '고객ID', '성별', '고객등급', '자택_시도', '수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '대출금액'])
    df = df.sort_values(by='기준년월', ascending=True).reset_index(drop=True)
    return df
    
M_CS_2022 = read_file(2022)

# 고객 id 
filtered_data = df[df['연령대'].isin(['20대', '30대'])]['고객ID'].unique()

# 20, 30대의 고객 ID만 뽑음
M_filtered_data = M_CS_2022[M_CS_2022['고객ID'].isin(filtered_data)]

# 수신잔고, sum(요구불 ~ 적립식)
M_filtered_data['수신잔고'] = M_filtered_data.groupby(['기준년월','고객ID'])[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금']].transform('sum').sum(axis=1)

# 지역구분 대구, 경상북도 => 1, 나머지 => 0
M_filtered_data['지역구분'] = np.where(M_filtered_data['자택_시도'].isin(['대구광역시', '경상북도']), 1, 0)                                                                

# 수신대비대출비율
M_filtered_data['수신대비대출비율'] = np.where(M_filtered_data['수신잔고'] != 0, M_filtered_data['대출금액'] / M_filtered_data['수신잔고'], 0)

result = M_filtered_data.merge(df[['고객ID', '연령대', '이탈여부']], on='고객ID', how='left')
result